In [1]:
import guide_tree_group
import guide_tree_compute
import blastn_process
import MHG_partition
import process_mhg
import consensus_mhg
import copy

In [64]:
test_genome_dir = "/shared/yy70/github/MHG-II/test_dataset/far_genomes/"
temp_genome_dir = 'temp_far/'
kmer_size = 16
thread = 1
mash_tree_path = "nj_far.newick"
blastn_dir = "temp_blastn_far/"
mhg_output_dir = "temp_far_mhg/"

In [3]:
# Concatenate fasta genomes to a directory; All assemblies are concat to the same file
accDic = guide_tree_compute.concat_fasta(test_genome_dir, temp_genome_dir)
# Mash to construct a distance matrix, and perform NJ. mash_tree_path contains the tree
guide_tree_compute.mash_distance_matrix_njtree(temp_genome_dir, mash_tree_path, kmer_size, thread)
# Reroot to obtain the lowest height reroot tree
rerooted_tree = guide_tree_group.shortest_reroot(mash_tree_path)
# Visited_node_MHG: internal node(key), MHG set(value); remaining_pair: list of remaining 
visited_node_MHG, remaining_pair = guide_tree_group.initial_taxa_internal(rerooted_tree)
# ready_MHG: None if next internal is NOT ready, otherwise key: internal node, value: MHG set

Sketching temp_far/cat.fa (provide sketch file made with "mash sketch" to skip)...done.


In [26]:
while remaining_pair:
    next_internal_ready_boolean, internal_node_taxa, ready_MHG_dict, visited_node_MHG, remaining_pair = guide_tree_group.give_me_the_next_visit(visited_node_MHG, remaining_pair)
    if next_internal_ready_boolean:
        #这里现在不行 assume的是两个children都是leaf taxa；如果一个children是internal node，blastn_next要重新写
        blastn_out_path = blastn_process.blastn_next(ready_MHG_dict, blastn_dir, temp_genome_dir)
        df, check_dict = MHG_partition.parseBlastXML(blastn_out_path)
        df = MHG_partition.trim_fully_contain(df, check_dict)
        mhg_list = MHG_partition.mhg(df, 2)
        if '|' not in internal_node_taxa:
            # Case 1: two children nodes are both leaf nodes
            pan_mhg_list = process_mhg.pangenome(mhg_list, accDic)
#             pan_mhg_list = process_mhg.pangenome_leaf(mhg_list, accDic)
        else:
            # Case 2: two children nodes have an internal node
            consensus_mhg_list = consensus_mhg.consensus_to_blocks(mhg_list, ready_MHG_dict)
            pan_mhg_list = pangenome_internal(consensus_mhg_list, accDic)
#             pan_mhg_list = process_mhg.pangenome_internal(consensus_mhg_list, accDic)
        refName_refBlcok_dict, ref_mhg_dict = process_mhg.mafft_consensus_mhg(pan_mhg_list, accDic)
        merged_internal_name = internal_node_taxa.replace(",","|")
        visited_node_MHG[merged_internal_name] = [refName_refBlcok_dict, ref_mhg_dict]
        # Convert consensus ref alignments to sequences, write to a new fasta and be ready for next blastn
        process_mhg.ref_alignment_to_fasta(merged_internal_name, temp_genome_dir, refName_refBlcok_dict)
    else:
        print("Next internal node is not ready")



Building a new DB, current time: 08/16/2022 11:09:10
New DB name:   /shared/yy70/github/MHG-II/meat/temp_far/G000412335|G000262305|G001277235|G000276705.fa
New DB title:  temp_far/G000412335|G000262305|G001277235|G000276705.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 30419 sequences in 0.541647 seconds.




Building a new DB, current time: 08/16/2022 11:09:12
New DB name:   /shared/yy70/github/MHG-II/meat/temp_far/G001598855|G000280495.fa
New DB title:  temp_far/G001598855|G000280495.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 11341 sequences in 0.231338 seconds.




2022-08-16 12:14:49,468 - root - INFO - start building dataframe containing pairwise blastn calls
2022-08-16 12:14:50,824 - root - INFO - start building an alignment graph
2022-08-16 12:14:51,675 - root - INFO - alignment graph finished building in time: 2.2073488235473633
2022-08-16 12:14:51,676 - root - INFO - starting traversing the alignment graph and MHG partition
2022-08-16 12:14:51,855 - root - INFO - Total 9462 cc are waiting to be visited
2022-08-16 12:14:51,856 - root - INFO - cc number 1 is on the show being visited
2022-08-16 12:14:51,901 - root - INFO - cc number 2 is on the show being visited
2022-08-16 12:14:51,902 - root - INFO - cc number 3 is on the show being visited
2022-08-16 12:14:51,902 - root - INFO - cc number 4 is on the show being visited
2022-08-16 12:14:51,903 - root - INFO - cc number 5 is on the show being visited
2022-08-16 12:14:51,903 - root - INFO - cc number 6 is on the show being visited
2022-08-16 12:14:51,904 - root - INFO - cc number 7 is on the 

2022-08-16 12:39:04,297 - root - INFO - cc number 96 is on the show being visited
2022-08-16 12:39:04,299 - root - INFO - cc number 97 is on the show being visited
2022-08-16 12:39:04,301 - root - INFO - cc number 98 is on the show being visited
2022-08-16 12:39:04,513 - root - INFO - cc number 99 is on the show being visited
2022-08-16 12:39:04,518 - root - INFO - cc number 100 is on the show being visited
2022-08-16 12:39:27,928 - root - INFO - cc number 101 is on the show being visited
2022-08-16 12:39:27,931 - root - INFO - cc number 102 is on the show being visited
2022-08-16 12:39:27,933 - root - INFO - cc number 103 is on the show being visited
2022-08-16 12:39:27,936 - root - INFO - cc number 104 is on the show being visited
2022-08-16 12:39:27,938 - root - INFO - cc number 105 is on the show being visited
2022-08-16 12:39:27,944 - root - INFO - cc number 106 is on the show being visited
2022-08-16 12:39:54,662 - root - INFO - cc number 107 is on the show being visited
2022-08-

2022-08-16 12:40:35,923 - root - INFO - cc number 195 is on the show being visited
2022-08-16 12:40:35,926 - root - INFO - cc number 196 is on the show being visited
2022-08-16 12:40:35,928 - root - INFO - cc number 197 is on the show being visited
2022-08-16 12:40:36,184 - root - INFO - cc number 198 is on the show being visited
2022-08-16 12:40:36,692 - root - INFO - cc number 199 is on the show being visited
2022-08-16 12:40:36,695 - root - INFO - cc number 200 is on the show being visited
2022-08-16 12:40:36,941 - root - INFO - cc number 201 is on the show being visited
2022-08-16 12:40:37,190 - root - INFO - cc number 202 is on the show being visited
2022-08-16 12:40:37,193 - root - INFO - cc number 203 is on the show being visited
2022-08-16 12:40:37,196 - root - INFO - cc number 204 is on the show being visited
2022-08-16 12:40:37,198 - root - INFO - cc number 205 is on the show being visited
2022-08-16 12:40:37,201 - root - INFO - cc number 206 is on the show being visited
2022

2022-08-16 12:40:58,681 - root - INFO - cc number 294 is on the show being visited
2022-08-16 12:40:58,933 - root - INFO - cc number 295 is on the show being visited
2022-08-16 12:40:58,936 - root - INFO - cc number 296 is on the show being visited
2022-08-16 12:40:58,939 - root - INFO - cc number 297 is on the show being visited
2022-08-16 12:40:59,190 - root - INFO - cc number 298 is on the show being visited
2022-08-16 12:40:59,193 - root - INFO - cc number 299 is on the show being visited
2022-08-16 12:40:59,447 - root - INFO - cc number 300 is on the show being visited
2022-08-16 12:40:59,713 - root - INFO - cc number 301 is on the show being visited
2022-08-16 12:40:59,716 - root - INFO - cc number 302 is on the show being visited
2022-08-16 12:40:59,719 - root - INFO - cc number 303 is on the show being visited
2022-08-16 12:40:59,722 - root - INFO - cc number 304 is on the show being visited
2022-08-16 12:40:59,975 - root - INFO - cc number 305 is on the show being visited
2022

...

2022-08-16 17:27:36,532 - root - INFO - cc number 9023 is on the show being visited
2022-08-16 17:27:36,631 - root - INFO - cc number 9024 is on the show being visited
2022-08-16 17:27:36,669 - root - INFO - cc number 9025 is on the show being visited
2022-08-16 17:27:36,706 - root - INFO - cc number 9026 is on the show being visited
2022-08-16 17:27:39,902 - root - INFO - cc number 9027 is on the show being visited
2022-08-16 17:27:43,142 - root - INFO - cc number 9028 is on the show being visited
2022-08-16 17:27:43,240 - root - INFO - cc number 9029 is on the show being visited
2022-08-16 17:27:43,278 - root - INFO - cc number 9030 is on the show being visited
2022-08-16 17:27:43,315 - root - INFO - cc number 9031 is on the show being visited
2022-08-16 17:27:43,352 - root - INFO - cc number 9032 is on the show being visited
2022-08-16 17:27:43,389 - root - INFO - cc number 9033 is on the show being visited
2022-08-16 17:27:43,426 - root - INFO - cc number 9034 is on the show being 

2022-08-16 17:28:40,692 - root - INFO - cc number 9121 is on the show being visited
2022-08-16 17:28:40,729 - root - INFO - cc number 9122 is on the show being visited
2022-08-16 17:28:40,767 - root - INFO - cc number 9123 is on the show being visited
2022-08-16 17:28:40,805 - root - INFO - cc number 9124 is on the show being visited
2022-08-16 17:28:40,849 - root - INFO - cc number 9125 is on the show being visited
2022-08-16 17:28:59,323 - root - INFO - cc number 9126 is on the show being visited
2022-08-16 17:28:59,360 - root - INFO - cc number 9127 is on the show being visited
2022-08-16 17:28:59,397 - root - INFO - cc number 9128 is on the show being visited
2022-08-16 17:28:59,434 - root - INFO - cc number 9129 is on the show being visited
2022-08-16 17:28:59,472 - root - INFO - cc number 9130 is on the show being visited
2022-08-16 17:28:59,509 - root - INFO - cc number 9131 is on the show being visited
2022-08-16 17:29:02,703 - root - INFO - cc number 9132 is on the show being 

2022-08-16 17:30:29,370 - root - INFO - cc number 9219 is on the show being visited
2022-08-16 17:30:29,407 - root - INFO - cc number 9220 is on the show being visited
2022-08-16 17:30:29,445 - root - INFO - cc number 9221 is on the show being visited
2022-08-16 17:30:32,723 - root - INFO - cc number 9222 is on the show being visited
2022-08-16 17:30:32,762 - root - INFO - cc number 9223 is on the show being visited
2022-08-16 17:30:38,548 - root - INFO - cc number 9224 is on the show being visited
2022-08-16 17:30:38,587 - root - INFO - cc number 9225 is on the show being visited
2022-08-16 17:30:38,624 - root - INFO - cc number 9226 is on the show being visited
2022-08-16 17:30:38,662 - root - INFO - cc number 9227 is on the show being visited
2022-08-16 17:30:38,699 - root - INFO - cc number 9228 is on the show being visited
2022-08-16 17:30:38,737 - root - INFO - cc number 9229 is on the show being visited
2022-08-16 17:30:38,775 - root - INFO - cc number 9230 is on the show being 

2022-08-16 17:34:12,844 - root - INFO - cc number 9317 is on the show being visited
2022-08-16 17:34:12,883 - root - INFO - cc number 9318 is on the show being visited
2022-08-16 17:34:12,921 - root - INFO - cc number 9319 is on the show being visited
2022-08-16 17:34:12,959 - root - INFO - cc number 9320 is on the show being visited
2022-08-16 17:34:15,649 - root - INFO - cc number 9321 is on the show being visited
2022-08-16 17:34:15,687 - root - INFO - cc number 9322 is on the show being visited
2022-08-16 17:34:15,725 - root - INFO - cc number 9323 is on the show being visited
2022-08-16 17:34:15,764 - root - INFO - cc number 9324 is on the show being visited
2022-08-16 17:34:15,802 - root - INFO - cc number 9325 is on the show being visited
2022-08-16 17:34:15,840 - root - INFO - cc number 9326 is on the show being visited
2022-08-16 17:34:15,878 - root - INFO - cc number 9327 is on the show being visited
2022-08-16 17:34:22,666 - root - INFO - cc number 9328 is on the show being 

2022-08-16 17:36:09,784 - root - INFO - cc number 9415 is on the show being visited
2022-08-16 17:36:09,822 - root - INFO - cc number 9416 is on the show being visited
2022-08-16 17:36:22,524 - root - INFO - cc number 9417 is on the show being visited
2022-08-16 17:36:22,563 - root - INFO - cc number 9418 is on the show being visited
2022-08-16 17:36:22,601 - root - INFO - cc number 9419 is on the show being visited
2022-08-16 17:36:22,639 - root - INFO - cc number 9420 is on the show being visited
2022-08-16 17:36:26,251 - root - INFO - cc number 9421 is on the show being visited
2022-08-16 17:36:26,289 - root - INFO - cc number 9422 is on the show being visited
2022-08-16 17:36:29,765 - root - INFO - cc number 9423 is on the show being visited
2022-08-16 17:36:29,804 - root - INFO - cc number 9424 is on the show being visited
2022-08-16 17:36:29,842 - root - INFO - cc number 9425 is on the show being visited
2022-08-16 17:36:33,189 - root - INFO - cc number 9426 is on the show being 

In [50]:
def mhg_output(mhg_output_dir, visited_node_MHG):
    os.mkdir(mhg_output_dir)
    for key in visited_node_MHG:
        if visited_node_MHG[key] == None:
            continue
        file_name = os.path.join(mhg_output_dir,f"{key}.txt")
        f = open(file_name,'w')
        ref_mhg_dict = visited_node_MHG[key][1]
        internal_node_mhg_list = list(ref_mhg_dict.values())
        print(key, len(internal_node_mhg_list))
        for mhg in internal_node_mhg_list:
            mhg_list = mhg.mhg_list
            block_string_list = [block.block_string for block in mhg_list]
            f.write(','.join(block_string_list)+'\n')
        f.close()
        
# mhg_output(mhg_output_dir, visited_node_MHG)

G000412335|G000262305 8957
G001277235|G000276705 11049
G001598855|G000280495 11341
G000412335|G000262305|G001277235|G000276705 30419
G001598855|G000280495|G000412335|G000262305|G001277235|G000276705 63327


In [15]:
import pandas as pd
import os 

def pangenome_internal(mhg_list, accDic):
    # Take >=2 blocks MHG list as input, intersect and complement with their whole genomes using
    # bedtools. Include all single-block MHGs and outputs the updated mhg_list which is like
    # a pangenome. Each block in each MHG is in the format: acc|start|end|direction
    pangenome_mhg_list = mhg_list[:]
    block_list = [b for m in pangenome_mhg_list for b in m]
    acc_list = [b.get_acc() for b in block_list]
    start_list = [b.get_start() for b in block_list]
    end_list = [b.get_end() for b in block_list]
    mhg_df = pd.DataFrame(list(zip(acc_list,start_list,end_list)), columns = ['id','start','end'])
    mhg_df.to_csv('mhg.bed',header=False, index = False,sep='\t')
    f = open('acc.bed','w')
    for acc in list(set(mhg_df['id'])):
        genome_length = len(accDic[acc])
        f.write(f"{acc}\t{1}\t{genome_length}\n")
    f.close()

    for i in range(len(pangenome_mhg_list)):
        pangenome_mhg_list[i] = [b.block_string for b in pangenome_mhg_list[i]]

    command = 'bedtools subtract -a acc.bed -b mhg.bed > single_block_mhg.bed'
    os.system(command)
    single_block_mhg_df = pd.read_csv('single_block_mhg.bed',delimiter='\t',header=None)
    single_block_mhg_df.columns = ['id','start','end']
    direction_list = list('+'*int(single_block_mhg_df.shape[0]))
    single_block_acc_list = list(single_block_mhg_df['id'])
    single_block_start_list = list(single_block_mhg_df['start'].astype(str))
    single_block_end_list = list(single_block_mhg_df['end'].astype(str))
    single_block_mhg_list = list(zip(single_block_acc_list,single_block_start_list,single_block_end_list,direction_list))
    single_block_mhg_list = [['|'.join(list(m))] for m in single_block_mhg_list]
    pangenome_mhg_list = pangenome_mhg_list + single_block_mhg_list

    return pangenome_mhg_list

pan_mhg_list = pangenome_internal(consensus_mhg_list, accDic)

In [51]:
[key for key in visited_node_MHG if visited_node_MHG[key] != None],remaining_pair
# visited_node_MHG['G000412335|G000262305|G001277235|G000276705']

(['G000412335|G000262305',
  'G001277235|G000276705',
  'G001598855|G000280495',
  'G000412335|G000262305|G001277235|G000276705',
  'G001598855|G000280495|G000412335|G000262305|G001277235|G000276705'],
 [])

In [77]:
# Validate gap ratio

import align_mhg
from io import StringIO
from itertools import groupby
import tqdm

taxa = [key for key in visited_node_MHG if visited_node_MHG[key] != None]
for key in taxa:
    if visited_node_MHG[key] == None:
        continue
    ref_mhg_dict = visited_node_MHG[key][1]
    internal_node_mhg_list = list(ref_mhg_dict.values())
    print(f"{key} has {len(internal_node_mhg_list)} mhg, and {len([m for m in internal_node_mhg_list if len(m.mhg_list) > 1])} mhg contains more than 1 block")
    for mhg in tqdm.tqdm_notebook(internal_node_mhg_list):
        mhg_list = mhg.mhg_list
        block_string_list = [block.block_string for block in mhg_list]
        alignment_string = align_mhg.print_alignment(temp_genome_dir,block_string_list)
        seqFile = StringIO(alignment_string)
        lines = (x[1] for x in groupby(seqFile, lambda line: line[0] == ">"))
        for seqName in lines:
            seqName = seqName.__next__()[1:].strip()
            seq = "".join(s.strip() for s in lines.__next__())
            gap_count = seq.count('-')
            gap_ratio = round(float(gap_count/len(seq)),3)
            if gap_ratio > 0.25:
                print(alignment_string)
                break

G000412335|G000262305 has 8957 mhg, and 3263 mhg contains more than 1 block


  0%|          | 0/8957 [00:00<?, ?it/s]

>G000262305|3495386|3495409|-
--------------------agtttctgatatcgttttcgcat-
>G000412335|403996|404040|+
ggtttcaggcttgcgcctgagagttttgatatcgtcttcgcatt
>G000412335|404129|404156|+
------------gtgcgaacgatatcttgcgtggttgtt-----

>G000262305|1619719|1619793|+
cgggaatagctcagttggtagagcacgaccttgccaaggtcggggtcgcgagttcgagtc
tcgtttcccgctcc-
>G000262305|3705282|3705356|-
cgggaatagctcagttggtagagcacgaccttgccaaggtcggggtcgcgagttcgagtc
tcgtttcccgctcc-
>G000262305|3705441|3705441|-
------------------------------------------------------------
---------------
>G000262305|3705569|3705642|-
-gggaatagctcagttggtagagcacgaccttgccaaggtcggggtcgcgagttcgagtc
tcgtttcccgctcc-
>G000412335|236835|236908|+
--ggaatagctcagttggtagagcacgaccttgccaaggtcggggtcgcgagttcgagtc
tcgtttcccgctcca
>G000412335|236946|237019|+
-gggaatagctcagttggtagagcacgaccttgccaaggtcggggtcgcgagttcgagtc
tcgtttcccgctcc-
>G000412335|237058|237132|+
-gggaatagctcagttggtagagcacgaccttgccaaggtcggggtcgcgagttcgagtc
tcgtttcccgctcca
>G000412335|2708533|2708607|-
cggga

  0%|          | 0/11049 [00:00<?, ?it/s]

>G000276705|3513164|3513185|+
--cggctatgcccccgttgtgcg
>G000276705|3622993|3623014|-
aagggctatgcgcccgtagta--
>G000276705|3994001|3994022|+
-agggtttcgccccggtcattc-
>G000276705|4399854|4399875|-
-gcggctacgcgcccgtcgttc-
>G001277235|1459117|1459138|-
cagggttacgcgcccgtcgtg--
>G001277235|1684422|1684422|+
-----------------------

>G000276705|1781515|1781536|+
aaatcttact-----ctattttcgat----
>G001277235|1335329|1335358|+
-aattttgttactcaccagccatgacgact

>G000276705|4245586|4245586|+
-------------------------------------------------------
>G000276705|4569938|4569992|+
-tgctgccgataacctgctgataccaggcaaaagatttcttacggctacgcgcca
>G000276705|5172943|5172943|-
-------------------------------------------------------
>G001277235|627579|627633|-
ttgctggcgacaacttttttataccaccagaacgatttcttacgcttgcgctcc-
>G001277235|1447793|1447847|+
-ggcttgcgatcacctgctggtaccacgcgaaactgcgctttttacgccgcgcca
>G001277235|1449200|1449200|+
-------------------------------------------------------
>G001277235|3662341|3662395|-
ttgctcgc

...

>G000280495|4259098|4259161|-
ccatggcgatcccgacatcggcctgtgccagcgcgggcgcgtcgtttatgccatcaccca
cca-
>G000262305|2923808|2923871|-
ccatggcgatcccgacttcggcctgggccagcgcgggcgcatcgttgatgccgtcaccaa
cca-
>G000276705|1435969|1436032|+
-catggcgatccccacatcggcctgcgccagcgcgggcgcatcgttgataccgtcgccga
tcat
>G000276705|1629314|1629314|-
------------------------------------------------------------
----
>G001277235|1189862|1189925|+
-catcgcgatacccacttccgcctgagccagcgccggggcgtcgttaatgccgtcgccaa
tcat
>G000412335|3366420|3366420|-
------------------------------------------------------------
----
>G001598855|2435196|2435259|+
-catggcgatacccacctcggcctgcgccagcgccggggcgtcgttgatgccgtcgccga
ccat
>G001598855|5023958|5024021|-
ccatggctatacccacgtcagccgctgccagcgccggggcatcattcacaccgtctccgg
cca-
>G000280495|3437816|3437879|+
-catggctatacccacgtcagccgctgccagcgccggggcatcattcacaccgtctccgg
ccat

>G000262305|3137704|3137736|+
cgccacggccacaacgataggcgggaccatct-----------------
>G000262305|3138011|3138042|+
-----------------ctggca-g

>G000280495|4775088|4775146|-
gtgctgctgttccaggatatcgcggcaatcccgctggtggcgatgattccgctgctgg-
>G000276705|887839|887839|+
-----------------------------------------------------------
>G000280495|427335|427335|+
-----------------------------------------------------------
>G001598855|4191326|4191326|+
-----------------------------------------------------------
>G001277235|744082|744082|+
-----------------------------------------------------------
>G001277235|4070586|4070645|-
gtgctgctgttccaggatctggcggtgatcccggcgctggcaatggtgccgctgctggc
>G000412335|487542|487542|+
-----------------------------------------------------------
>G000412335|4277113|4277171|-
gtgttgttattccaggatctcgcggttattcctgcgctggcgttggtgcctttgctgg-
>G001598855|1647485|1647485|+
-----------------------------------------------------------
>G000262305|297573|297632|+
gtgttgctgtttcaggatctggcggtgatcccggcagtggcgctggtgccgctgctggc

>G001277235|142160|142181|+
acagcgtcttcgacgactgcg------------
>G001277235|145089|145110|+
acagcgtcttcgacgactg

>G001598855|2054857|2054973|-
gcagaccatcagcgcgctgggaaccctggcggccgggctggcgggagggctcactggagg
cagcaccgcagatgcagtggccggggcgcaggccgggaagaatgcggttgagaata-
>G001598855|5605976|5606092|-
gcagaccatcagtgcgctgggaaccctggcggccgggctggcgggagggctcactggcgg
cagcaccgcagatgcagtggccggggcgcaggccgggaagaatgcggttgagaata-
>G000262305|976733|976849|+
-cagacggtcagcgtgctgggtacgctggcggcgggcctggctggaggcgtttccggagg
cggtagcgtatctgccctggcaggggcagaggcaggaaggaatgcggcggagaataa
>G000262305|1779534|1779534|+
------------------------------------------------------------
---------------------------------------------------------
>G000262305|1782721|1782721|+
------------------------------------------------------------
---------------------------------------------------------
>G000262305|1788617|1788617|+
------------------------------------------------------------
---------------------------------------------------------
>G000262305|1791738|1791738|+
------------------------------------------------------------
-----------------

>G000276705|5007655|5007655|+
---------------------------
>G001277235|3212644|3212669|-
cgatttcagcaccacgaacgctgcc--
>G001277235|3595463|3595489|+
-gccttcagcaccgcaaatcccgcctg
>G000412335|3296089|3296114|-
-gacttcaacaccacaaatgcggcct-
>G000412335|3780512|3780537|+
-gctttaagtactgcaaacccggcct-
>G000280495|977139|977164|-
-gctttcagcaccgcaaaaccggcct-
>G001598855|2501820|2501845|-
-gccttcagcaccgcaaacccggcct-
>G000280495|1490267|1490292|+
-gattttagcaccacaaacgcggcct-
>G001598855|5263431|5263456|+
-gacttcagtaccacgaatgccgcct-
>G000262305|829354|829354|-
---------------------------
>G000262305|1121097|1121123|+
-gatttcagcaccacgaaagcggcctg

>G000280495|4223798|4223819|+
--tctcgcgcaaagttatcggct
>G000280495|4514226|4514247|-
-ttctcacgcaaaattatcggc-
>G000280495|4924903|4924924|+
--tctcacgcaaaattatcggct
>G000280495|5924586|5924607|+
--tctcacgcaaaattatcggct
>G000276705|6129900|6129921|-
-ttctcgcgcaaagttattggc-
>G001598855|746684|746705|-
-tggtcccgtgcggtcattggc-
>G001598855|901863|901884|-
-tggtcacgtgcggt

>G000276705|4233366|4233385|+
--------cgcgcagatgcagattgag
>G000280495|1814180|1814200|-
cgttttgacatgcagatcga-------
>G001598855|256940|256960|-
------ttcgcgcagagtaagattaa-

>G000412335|2671084|2671107|+
cggcagagctgacatcgccccgt---------
>G001277235|2716672|2716695|+
cggcatagcggccattgcgccgt---------
>G000280495|1249527|1249550|+
---------gggcattttgctgaaagcggcat



In [90]:
import collections

taxa = [key for key in visited_node_MHG if visited_node_MHG[key] != None]
for key in taxa:
    coverage = collections.defaultdict(int)
    if visited_node_MHG[key] == None:
        continue
    ref_mhg_dict = visited_node_MHG[key][1]
    internal_node_mhg_list = list(ref_mhg_dict.values())
    print(f"{key} has {len(internal_node_mhg_list)} mhg, and {len([m for m in internal_node_mhg_list if len(m.mhg_list) > 1])} mhg contains more than 1 block")
    for mhg in internal_node_mhg_list:
        mhg_list = mhg.mhg_list
        if len(mhg_list) < 2:
            continue
        block_string_list = [block.block_string for block in mhg_list]
        for b_string in block_string_list:
            b_string = b_string.split('|')
            coverage[b_string[0]] += (int(b_string[2]) - int(b_string[1]))
    for key in coverage:
        print(f"{key} covered {coverage[key]} bp out of {len(accDic[key])} bp; ratio {round((coverage[key]/len(accDic[key])),2)}")
    print("")

G000412335|G000262305 has 8957 mhg, and 3263 mhg contains more than 1 block
G000262305 covered 2273081 bp out of 4158725 bp; ratio 0.55
G000412335 covered 2233120 bp out of 4886981 bp; ratio 0.46

G001277235|G000276705 has 11049 mhg, and 4054 mhg contains more than 1 block
G000276705 covered 2739187 bp out of 6561678 bp; ratio 0.42
G001277235 covered 2662088 bp out of 4628405 bp; ratio 0.58

G001598855|G000280495 has 11341 mhg, and 4285 mhg contains more than 1 block
G000280495 covered 2943718 bp out of 6122453 bp; ratio 0.48
G001598855 covered 2901358 bp out of 5662775 bp; ratio 0.51

G000412335|G000262305|G001277235|G000276705 has 30419 mhg, and 7600 mhg contains more than 1 block
G000262305 covered 2363093 bp out of 4158725 bp; ratio 0.57
G000412335 covered 2646781 bp out of 4886981 bp; ratio 0.54
G000276705 covered 2816664 bp out of 6561678 bp; ratio 0.43
G001277235 covered 2448093 bp out of 4628405 bp; ratio 0.53

G001598855|G000280495|G000412335|G000262305|G001277235|G000276705 h

In [51]:
df[df['alignmentLength'] > 100]

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,"ref_0:G001277235,G000276705","ref_5061:G000412335,G000262305",80.844,569,109,0,1,569,19,587,1.908050e-131,465,TTGTGGGCCTTGCCAACCCCGGCGCGGAATACGCCGCTACCCGCCA...,TTGTCGGCCTGGCCAACCCGGGCGCTGAGTATGCCGCCACCCGCCA...
61,"ref_1:G001277235,G000276705","ref_3938:G000412335,G000262305",86.375,411,56,0,1,411,7,417,4.478580e-132,467,GAGGTAGCAAGTGACCATTGCTATTGTAATAGGCACACATGGTTGG...,GAGGTAGCAAGTGACTATTGCTATTGTCATAGGCACACATGGCTGG...
181,"ref_5:G001277235,G000276705","ref_5139:G000412335,G000262305",85.030,167,25,0,2,168,518,352,3.434130e-45,177,TCCCAGTCAGTCACTTCTTTCTCCGCTTGGTCTTTGGCGTAACCGT...,TCCCAGTCCTTCACTTCTTTCTCTGCCTGGTCTTTGGCATAGCCGT...
220,"ref_7:G001277235,G000276705","ref_3696:G000412335,G000262305",81.319,1638,303,3,5,1639,91,1728,0.000000e+00,1365,CGACGCCAGCATGCCGCGCAGGATGTTCAGCTCCTGGCTTTCCGGG...,CGACGCCAGAATGCCGCGCAAAATATTTAACTCCTCACTTTCCGGG...
370,"ref_10:G001277235,G000276705","ref_3241:G000412335,G000262305",80.389,566,106,5,2,567,1034,1594,2.392630e-123,439,ATGGCCTATAACCGTCTGAACCAGCGCGTGAACAAGCTGGAGCTGA...,ATGGCGTACAACCGCCTCAGCCAGCGGGTGAACAAACTGGAACTTA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829864,"ref_15640:G001277235,G000276705","ref_7295:G000412335,G000262305",73.554,121,32,0,35817,35937,4689,4809,1.128050e-03,48,GTGAAATCGCACACCTGGTGATTGATTCCACCGGGCTGAAGGTCTT...,GTGAAATCGTCCATTTGGTAATAGATTCAGCAGGGGTGAAAGTCGT...
830047,"ref_15646:G001277235,G000276705","ref_9814:G000412335,G000262305",80.977,573,107,2,1760,2330,3708,3136,3.042830e-130,464,CTCACGGACACCCATGAAAGCCGCCAGCTTATGGTTTTGTCGCTGG...,CTCACCGGCACCCACGAAAGCCGCCAGCTGATGGTACTGTCGCTGG...
830087,"ref_15647:G001277235,G000276705","ref_9814:G000412335,G000262305",77.677,775,173,0,179,953,2880,2106,2.597310e-139,492,CTGACCCGCCTCAGCCATAGCCAGTGCGAACTGCTGGCCAGCGATG...,CTGACACACATCAGCCACAGCCAGTGCGAGCTACAGGCCAGTGACG...
830215,"ref_15650:G001277235,G000276705","ref_2853:G000412335,G000262305",85.102,537,80,0,17875,18411,1,537,1.957820e-161,571,CCGTGTCTCATGTCGCGCGCCTGCATGGCATCAATGCAAACCAGAT...,CCGTGTCCCACGTTGCCCGTCTGCATGGCATAAATGCCAACCAGAT...


In [65]:
df[df['alignmentLength'] > 60]

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,"ref_0:G001277235,G000276705","ref_5061:G000412335,G000262305",80.844,569,109,0,1,569,19,587,1.908050e-131,465,TTGTGGGCCTTGCCAACCCCGGCGCGGAATACGCCGCTACCCGCCA...,TTGTCGGCCTGGCCAACCCGGGCGCTGAGTATGCCGCCACCCGCCA...
61,"ref_1:G001277235,G000276705","ref_3938:G000412335,G000262305",86.375,411,56,0,1,411,7,417,4.478580e-132,467,GAGGTAGCAAGTGACCATTGCTATTGTAATAGGCACACATGGTTGG...,GAGGTAGCAAGTGACTATTGCTATTGTCATAGGCACACATGGCTGG...
181,"ref_5:G001277235,G000276705","ref_5139:G000412335,G000262305",85.030,167,25,0,2,168,518,352,3.434130e-45,177,TCCCAGTCAGTCACTTCTTTCTCCGCTTGGTCTTTGGCGTAACCGT...,TCCCAGTCCTTCACTTCTTTCTCTGCCTGGTCTTTGGCATAGCCGT...
220,"ref_7:G001277235,G000276705","ref_3696:G000412335,G000262305",81.319,1638,303,3,5,1639,91,1728,0.000000e+00,1365,CGACGCCAGCATGCCGCGCAGGATGTTCAGCTCCTGGCTTTCCGGG...,CGACGCCAGAATGCCGCGCAAAATATTTAACTCCTCACTTTCCGGG...
312,"ref_9:G001277235,G000276705","ref_4142:G000412335,G000262305",87.805,82,10,0,126,207,91,10,6.079940e-22,100,TATCGCCGTTGTTCAGCGCTTCGTTCGGCAGAACGTAGTCGTTAAA...,TATCGCCTTTGCTCAGCGCTTCGTTAGGCAGAACGTAATCGGTAAA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829887,"ref_15641:G001277235,G000276705","ref_2882:G000412335,G000262305",75.269,93,23,0,113,205,393,301,2.088570e-05,46,AAATATACACTAACGCCAAAGCTGTGGGATTATCTCTATAAGTACG...,AAATATATTCTGACGCCGACCCTGTGGCACTACCTCTACAAGTACG...
830047,"ref_15646:G001277235,G000276705","ref_9814:G000412335,G000262305",80.977,573,107,2,1760,2330,3708,3136,3.042830e-130,464,CTCACGGACACCCATGAAAGCCGCCAGCTTATGGTTTTGTCGCTGG...,CTCACCGGCACCCACGAAAGCCGCCAGCTGATGGTACTGTCGCTGG...
830087,"ref_15647:G001277235,G000276705","ref_9814:G000412335,G000262305",77.677,775,173,0,179,953,2880,2106,2.597310e-139,492,CTGACCCGCCTCAGCCATAGCCAGTGCGAACTGCTGGCCAGCGATG...,CTGACACACATCAGCCACAGCCAGTGCGAGCTACAGGCCAGTGACG...
830215,"ref_15650:G001277235,G000276705","ref_2853:G000412335,G000262305",85.102,537,80,0,17875,18411,1,537,1.957820e-161,571,CCGTGTCTCATGTCGCGCGCCTGCATGGCATCAATGCAAACCAGAT...,CCGTGTCCCACGTTGCCCGTCTGCATGGCATAAATGCCAACCAGAT...


In [55]:
df[df['alignmentLength'] > 20]

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,"ref_0:G001277235,G000276705","ref_5061:G000412335,G000262305",80.844,569,109,0,1,569,19,587,1.908050e-131,465,TTGTGGGCCTTGCCAACCCCGGCGCGGAATACGCCGCTACCCGCCA...,TTGTCGGCCTGGCCAACCCGGGCGCTGAGTATGCCGCCACCCGCCA...
4,"ref_0:G001277235,G000276705","ref_11939:G000412335,G000262305",88.462,26,3,0,333,358,81,106,3.218180e-01,33,CTGAAAGACATCATCAGCAAGCTTGG,CTGAAAAACATCATCAGCATGATTGG
13,"ref_0:G001277235,G000276705","ref_5879:G000412335,G000262305",90.909,22,2,0,401,422,9208,9229,1.220100e+00,31,TCCGGGCGATAAAAATAAAGTT,TCCAGGCGATAAAAATTAAGTT
14,"ref_0:G001277235,G000276705","ref_5720:G000412335,G000262305",90.909,22,2,0,69,90,2438,2459,1.220100e+00,31,GATTTACTGGCTGATCGCCACC,GATTTACTGGCCGATAGCCACC
17,"ref_0:G001277235,G000276705","ref_190:G000412335,G000262305",88.000,25,3,0,387,411,173,197,1.220100e+00,31,CTGGGCATCGGCCATCCGGGCGATA,CTGGACAACGGCCATCCGGTCGATA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830266,"ref_15650:G001277235,G000276705","ref_3094:G000412335,G000262305",84.848,33,5,0,21611,21643,1748,1716,5.933060e+00,35,TCCGTTCTCTGACGGAAGCCATCGACACCACCA,TCCGTGCTCTGACGGAAGAGACCGGCACCACCA
830267,"ref_15650:G001277235,G000276705","ref_2365:G000412335,G000262305",91.667,24,2,0,4955,4978,3059,3082,5.933060e+00,35,GGCGCTCAGACGCTGGCTGGTCAG,GGCGCTCAGCAGCTGGCTGGTCAG
830268,"ref_15650:G001277235,G000276705","ref_1217:G000412335,G000262305",95.238,21,1,0,2213,2233,203,183,5.933060e+00,35,ATCGGCCAGGCAATCAGCGCG,ATCTGCCAGGCAATCAGCGCG
830270,"ref_15650:G001277235,G000276705","ref_1039:G000412335,G000262305",91.667,24,2,0,3044,3067,566,589,5.933060e+00,35,CGCGGGCGGCAAGCGCCATCGCTT,CGCGGGCGCCCAGCGCCATCGCTT


In [ ]:
dict_1 = visited_node_MHG['G000412335,G000262305'][0]
dict_2 = visited_node_MHG['G000412335,G000262305'][1]
for ref_name in list(dict_1.keys())[:100]:
    ref_block = dict_1[ref_name]
    mhg = dict_2[ref_block].mhg_list
    print("ref")
    print(ref_block.alignment)
    for b in mhg:
        print(b.alignment)
    print("")